In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv(r'C:\Users\DELL\OneDrive\Desktop\OnlineRetail.csv')
def summary(df):
        display(df.head())
        print('-'*100)
        display(df.info())
        print('-'*100)
        display(df.describe([0.01,0.25,0.50,0.75,0.99]))
summary(df)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df = df[(df.Quantity>0) & (df.UnitPrice> 0)]
df = df[~df['StockCode'].str.contains('C')]
df = df.drop_duplicates()
df.isnull().sum()
df.dropna(inplace=True)
df['Total_Price'] = df['UnitPrice']*df['Quantity']
df['InvoiceDate'].max() # let's see the latest day 
now =  dt.datetime(2011,12,10)
rfm = df.groupby('CustomerID').agg({'InvoiceDate' : lambda day : (now - day.max()).days,
                               'InvoiceNo': lambda num : len(num),
                              'Total_Price': lambda price : price.sum()
                                                                 })
col_list = ['Recency','Frequency','Monetary']
rfm.columns = col_list
rfm["R"] = pd.qcut(rfm["Recency"],5,labels=[5,4,3,2,1])
rfm["F"] = pd.qcut(rfm["Frequency"],5,labels=[1,2,3,4,5])
rfm["M"] = pd.qcut(rfm["Monetary"],5,labels=[1,2,3,4,5])
rfm["RFM_Score"] = rfm["R"].astype(str) +rfm["F"].astype(str) + rfm["M"].astype(str)
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}
rfm['Segment'] = rfm['R'].astype(str) + rfm['F'].astype(str)
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
rfm.head()
rfm.groupby('Segment').mean().sort_values('Monetary')


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


None

----------------------------------------------------------------------------------------------------


,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
1%,-2.000000,0.190000,12415.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
99%,100.000000,18.000000,18212.000000
max,80995.000000,38970.000000,18287.000000


,Recency,Frequency,Monetary
Segment,,,
Promising,22.888889,7.777778,418.106481
About to Sleep,52.134969,15.947853,448.272638
Hibernating,209.496317,13.659300,531.554725
Need Attention,51.975490,41.225490,821.920392
Potential Loyalists,15.643145,34.723790,897.880022
At Risk,163.561151,56.625899,1000.168022
Can't Loose,144.851852,177.493827,2379.549889
Loyal Customers,33.696587,153.356511,2813.549494
New Customers,7.047619,7.650794,3352.233333
